In [1]:

from VariationalBayes.MultinomialParams import \
    constrain_simplex_vector, constrain_hess_from_moment
import autograd.numpy as np

from autograd import grad, jacobian, hessian
import numpy.testing as np_test
import matplotlib.pyplot as plt


k = 3
free_param = np.arange(0., float(k), 1.)
z = constrain_simplex_vector(free_param)
get_constrain_hess = hessian(constrain_simplex_vector)

target_hess = get_constrain_hess(free_param)
hess = constrain_hess_from_moment(z)

print(target_hess.shape)
print(hess.shape)

# print(target_hess)
# print(hess)
diff = hess - target_hess
diff[np.abs(diff) < 1e-6] = 0
print(diff)
print(z)


(4, 3, 3)
(4, 3, 3)
[[[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]
[ 0.08259454  0.08259454  0.22451524  0.61029569]
